In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 7.6 MB/s eta 0:00:00


In [3]:
# ===================================================================
#  Library
# ===================================================================
import pandas as pd
from sklearn.metrics import mean_absolute_percentage_error

import warnings
warnings.simplefilter("ignore")

import numpy as np
from scipy.optimize import minimize
import optuna
import lightgbm as lgb

In [4]:
# ===================================================================
#  CFG
# ===================================================================
class CFG:
    seed = 42
    n_seeds = 3
    n_trials = 2000
    save_dir = "/content/drive/MyDrive/Colab Notebooks/signate2023/exp/"
    data_dir = "/content/drive/MyDrive/Colab Notebooks/signate2023/"
    filename = "exp00066"

In [5]:
# ===================================================================
#  Utils
# ===================================================================
def get_score(y_true, y_pred):
    """get MAPE score"""
    score = mean_absolute_percentage_error(y_true, y_pred)
    return score * 100

In [6]:
kun_exp00061_oof_pred= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/kun_exp00061_oof_pred.csv').rename(columns={'oof_pred': 'kun_pred_0'})

In [7]:
kun_exp00062_oof_pred = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/kun_exp00062_oof_pred.csv').rename(columns={'oof_pred': 'kun_pred_1'})

In [8]:
kun_oof =  kun_exp00061_oof_pred.merge(kun_exp00062_oof_pred, on='id')

In [9]:
kun_oof

,id,kun_pred_0,kun_pred_1
0,0,8365.577188,8227.1710
1,1,4005.751997,3704.7322
2,2,2941.156625,2834.8005
3,3,8067.291115,8023.7600
4,4,4383.119601,4103.2240
...,...,...,...
27527,27527,12749.206239,14307.2880
27528,27528,6826.593043,6751.7050
27529,27529,14849.231432,15627.8750
27530,27530,6763.711515,6789.5425


In [10]:
oof_df_exp072 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/oof_df_exp072.csv')[['id', 'pred']].rename(columns={'pred': 'pred_0'})
oof_df_exp073 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/oof_df_exp073.csv')[['id', 'pred']].rename(columns={'pred': 'pred_1'})
oof_df_exp074 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/oof_df_exp074.csv')[['id', 'pred']].rename(columns={'pred': 'pred_2'})
oof_df_exp075 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/oof_df_exp075.csv')[['id', 'pred']].rename(columns={'pred': 'pred_3'})
oof_df_exp076 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/oof_df_exp076.csv')[['id', 'pred']].rename(columns={'pred': 'pred_4'})

In [11]:
yuji_oof = pd.concat([oof_df_exp072, oof_df_exp073['pred_1'], oof_df_exp074['pred_2'], oof_df_exp075['pred_3'], oof_df_exp076['pred_4']],axis=1)

In [12]:
yuji_oof

,id,pred_0,pred_1,pred_2,pred_3,pred_4
0,0,8349.927994,8347.671992,8374.812648,8384.157015,8368.712800
1,1,3985.761719,4009.450083,3988.691655,3988.850911,3991.109782
2,2,2918.185395,2933.057596,2921.001787,2921.707668,2917.474388
3,3,8048.014373,8064.602372,8060.942061,8061.291081,8059.011798
4,4,4406.907983,4421.068587,4408.700347,4412.510810,4412.382296
...,...,...,...,...,...,...
27527,27527,12826.898779,12856.691563,12754.003557,12773.660474,12764.761508
27528,27528,6756.877560,6809.333388,6837.699939,6833.606242,6844.755058
27529,27529,14832.178562,14826.650466,14752.448736,14770.507371,14743.790056
27530,27530,6768.910608,6758.070369,6761.688384,6770.547369,6763.082951


In [13]:
kun_exp00061 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/kun_exp00061.csv', header=None).rename(columns={0:'id', 1: 'kun_pred_0'})
kun_exp00062 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/kun_exp00062.csv', header=None).rename(columns={0:'id', 1: 'kun_pred_1'})

In [14]:
kun_test =  kun_exp00061.merge(kun_exp00062, on='id')

In [15]:
kun_test

,id,kun_pred_0,kun_pred_1
0,27532,9222.758846,8606.683365
1,27533,5569.564552,5412.844072
2,27534,5680.045525,5585.359034
3,27535,18570.819391,19174.202965
4,27536,4246.374616,4395.118442
...,...,...,...
27532,55064,13650.838759,15507.759200
27533,55065,8920.871749,8243.670506
27534,55066,5736.867370,5585.359034
27535,55067,4867.610997,4624.595974


In [16]:
exp072 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/exp072.csv', header=None).rename(columns={0:'id', 1: 'pred_0'})
exp073 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/exp073.csv', header=None).rename(columns={0:'id', 1: 'pred_1'})
exp074 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/exp074.csv', header=None).rename(columns={0:'id', 1: 'pred_2'})
exp075 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/exp075.csv', header=None).rename(columns={0:'id', 1: 'pred_3'})
exp076 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/exp076.csv', header=None).rename(columns={0:'id', 1: 'pred_4'})

In [17]:
yuji_test = pd.concat([exp072, exp073['pred_1'], exp074['pred_2'], exp075['pred_3'], exp076['pred_4']],axis=1)

In [18]:
yuji_test

,id,pred_0,pred_1,pred_2,pred_3,pred_4
0,27532,9219.200678,9262.398861,9209.845453,9211.657631,9201.403646
1,27533,5555.610927,5570.955783,5546.121886,5550.776992,5544.537385
2,27534,5653.424701,5681.220795,5667.925276,5667.435425,5669.104669
3,27535,18752.937246,18712.897270,18594.433100,18620.599651,18594.588899
4,27536,4234.809532,4254.344876,4254.846798,4256.454343,4253.998513
...,...,...,...,...,...,...
27532,55064,13668.394800,13729.828156,13595.761827,13602.629201,13604.884197
27533,55065,8905.917019,8909.020788,8889.727327,8897.219594,8872.642211
27534,55066,5724.581797,5746.568914,5738.542173,5737.161601,5741.530500
27535,55067,4860.418758,4865.465520,4861.010701,4864.039152,4861.130000


In [19]:
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/train.csv')
df = pd.concat([kun_oof.merge(yuji_oof, on='id'), df_train['price']], axis=1)
test = kun_test.merge(yuji_test, on='id')

In [20]:
df

,id,kun_pred_0,kun_pred_1,pred_0,pred_1,pred_2,pred_3,pred_4,price
0,0,8365.577188,8227.1710,8349.927994,8347.671992,8374.812648,8384.157015,8368.712800,27587
1,1,4005.751997,3704.7322,3985.761719,4009.450083,3988.691655,3988.850911,3991.109782,4724
2,2,2941.156625,2834.8005,2918.185395,2933.057596,2921.001787,2921.707668,2917.474388,10931
3,3,8067.291115,8023.7600,8048.014373,8064.602372,8060.942061,8061.291081,8059.011798,16553
4,4,4383.119601,4103.2240,4406.907983,4421.068587,4408.700347,4412.510810,4412.382296,5158
...,...,...,...,...,...,...,...,...,...
27527,27527,12749.206239,14307.2880,12826.898779,12856.691563,12754.003557,12773.660474,12764.761508,32212
27528,27528,6826.593043,6751.7050,6756.877560,6809.333388,6837.699939,6833.606242,6844.755058,5400
27529,27529,14849.231432,15627.8750,14832.178562,14826.650466,14752.448736,14770.507371,14743.790056,22227
27530,27530,6763.711515,6789.5425,6768.910608,6758.070369,6761.688384,6770.547369,6763.082951,3054


In [21]:
test

,id,kun_pred_0,kun_pred_1,pred_0,pred_1,pred_2,pred_3,pred_4
0,27532,9222.758846,8606.683365,9219.200678,9262.398861,9209.845453,9211.657631,9201.403646
1,27533,5569.564552,5412.844072,5555.610927,5570.955783,5546.121886,5550.776992,5544.537385
2,27534,5680.045525,5585.359034,5653.424701,5681.220795,5667.925276,5667.435425,5669.104669
3,27535,18570.819391,19174.202965,18752.937246,18712.897270,18594.433100,18620.599651,18594.588899
4,27536,4246.374616,4395.118442,4234.809532,4254.344876,4254.846798,4256.454343,4253.998513
...,...,...,...,...,...,...,...,...
27532,55064,13650.838759,15507.759200,13668.394800,13729.828156,13595.761827,13602.629201,13604.884197
27533,55065,8920.871749,8243.670506,8905.917019,8909.020788,8889.727327,8897.219594,8872.642211
27534,55066,5736.867370,5585.359034,5724.581797,5746.568914,5738.542173,5737.161601,5741.530500
27535,55067,4867.610997,4624.595974,4860.418758,4865.465520,4861.010701,4864.039152,4861.130000


In [22]:
def get_custom_cv(df, n_splits=5):
    # priceを小さい順に各foldに振り分ける
    df = df.sort_values(by="price", ignore_index=True)
    df["fold"] = [i for i in range(n_splits)] * (df.shape[0] // n_splits) \
                + [i for i in range(df.shape[0] % n_splits)]
    df = df.sort_values(by="id", ignore_index=True)


    for fold in range(n_splits):
        train_idx = df[df["fold"] != fold].index
        valid_idx = df[df["fold"] == fold].index
        yield train_idx, valid_idx

n_splits = 8
cv = list(get_custom_cv(df, n_splits))

In [23]:
def fit_lgbm(X, y, test_data, cv, categorical_cols: list = None, params: dict = None, verbose: int = 50):
    if params is None:
        params = {}

    models = []
    n_records = len(X)
    oof_pred = np.zeros((n_records,), dtype=np.float32)
    test_list = []

    for i, (idx_train, idx_valid) in enumerate(cv):
        X_tr, y_tr = X.iloc[idx_train], y.iloc[idx_train]
        X_val, y_val = X.iloc[idx_valid], y.iloc[idx_valid]
        transformed_test_data = test_data.copy()


        drop_cols = ['id', 'price']
        X_tr.drop(drop_cols, axis=1, inplace=True)
        X_val.drop(drop_cols, axis=1, inplace=True)
        transformed_test_data.drop(['id'], axis=1, inplace=True)
        test_list.append(transformed_test_data)


        model = lgb.LGBMRegressor(**params)
        model.fit(
            X_tr,
            y_tr,
            categorical_feature=categorical_cols,
            eval_set=[(X_val, y_val)],
            callbacks=[lgb.early_stopping(100, verbose=verbose)],
        )
        pred_i = model.predict(X_val)
        oof_pred[idx_valid] = pred_i
        models.append(model)
        score = mean_absolute_percentage_error(y_val, pred_i)
        print(f" - fold{i + 1} - {score:.4f}")

    score = mean_absolute_percentage_error(y, oof_pred)
    print("=" * 50)
    print(f"FINISHI: Whole Score: {score:.4f}")
    return score, oof_pred, models, test_list




# optunaを使ったパラメータチューニング
def tuning(df, cv):
     def objective(trial):
         max_depth = trial.suggest_int("max_depth", 1, 10)
         params = {
              "max_depth": max_depth,
              "n_estimators": trial.suggest_int('n_estimators', 100, 50000),
              "learning_rate": trial.suggest_float("learning_rate", 0.01, 1.0),
              "num_leaves": trial.suggest_int("num_leaves", 2, 2**max_depth),
              "colsample_bytree": trial.suggest_uniform("colsample_bytree", 0.1, 1.0),
              "subsample": trial.suggest_float("subsample", 0.01, 1.0),
              "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 0, 300),
              "max_bin": trial.suggest_int("max_bin", 32, 512),
              "reg_lambda": trial.suggest_float("reg_lambda", 1e-10, 100.0),
              "task":"train",
              "objective": "mape",
              "boosting":"gbdt",
              "metric": 'mape',
              "importance_type": "gain",
              "seed" : 88,
              "verbosity": -1,


         }

         score, _, _, _ = fit_lgbm(
             X=df, y=df["price"], test_data=test, cv=cv,  categorical_cols=[], params=params, verbose=-1
         )
         return score

     study = optuna.create_study(direction="minimize")
     study.optimize(objective, n_trials=100)
     print("Number of finished trials:", len(study.trials))
     print("Best trial:", study.best_trial.params)
     return study.best_trial.params

In [24]:
lgbm_params = tuning(df, cv)

lgbm_params.update(
    {
    "task":"train",
    "objective": "mape",
    "boosting":"gbdt",
    "metric": 'mape',
    "importance_type": "gain",
    "seed" : 88,
    "verbosity": -1,
    }
)


# training
score, oof_lgb, models_lgb, transformed_test_data_list = fit_lgbm(
    X=df,
    y=df["price"],
    test_data=test,
    categorical_cols=[],
    params=lgbm_params,
    cv=cv,
    verbose=False,
)

[I 2023-08-24 04:14:52,918] A new study created in memory with name: no-name-defca951-1160-423c-b259-a6d1ae1e03ce


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[12]	valid_0's mape: 0.432174
 - fold1 - 0.4322
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[11]	valid_0's mape: 0.445551
 - fold2 - 0.4456
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[14]	valid_0's mape: 0.433841
 - fold3 - 0.4338
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[13]	valid_0's mape: 0.44606
 - fold4 - 0.4461
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[13]	valid_0's mape: 0.433201
 - fold5 - 0.4332
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[19]	valid_0's mape: 0.429111
 - fold6 - 0.4291
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[8]	valid_0's mape: 0.446194
 - fo

[I 2023-08-24 04:15:01,324] Trial 0 finished with value: 0.4379271290644998 and parameters: {'max_depth': 9, 'n_estimators': 42516, 'learning_rate': 0.2753845052379479, 'num_leaves': 451, 'colsample_bytree': 0.5642840764514935, 'subsample': 0.17860365367091588, 'min_data_in_leaf': 72, 'max_bin': 180, 'reg_lambda': 9.013620910696966}. Best is trial 0 with value: 0.4379271290644998.


Early stopping, best iteration is:
[10]	valid_0's mape: 0.437283
 - fold8 - 0.4373
FINISHI: Whole Score: 0.4379
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[4]	valid_0's mape: 0.431934
 - fold1 - 0.4319
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[3]	valid_0's mape: 0.443178
 - fold2 - 0.4432
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[44]	valid_0's mape: 0.432415
 - fold3 - 0.4324
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[3]	valid_0's mape: 0.447178
 - fold4 - 0.4472
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[4]	valid_0's mape: 0.430882
 - fold5 - 0.4309
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[3]	valid_0's mape: 0.429401
 - fold6 - 0.4294
Training until validati

[I 2023-08-24 04:15:07,740] Trial 1 finished with value: 0.4372597914156896 and parameters: {'max_depth': 8, 'n_estimators': 39192, 'learning_rate': 0.7320968182151317, 'num_leaves': 59, 'colsample_bytree': 0.6876609761095133, 'subsample': 0.6567712443281953, 'min_data_in_leaf': 198, 'max_bin': 119, 'reg_lambda': 74.71819994853504}. Best is trial 1 with value: 0.4372597914156896.


...

Early stopping, best iteration is:
[142]	valid_0's mape: 0.435837
 - fold8 - 0.4358
FINISHI: Whole Score: 0.4357
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[194]	valid_0's mape: 0.43018
 - fold1 - 0.4302
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[97]	valid_0's mape: 0.442797
 - fold2 - 0.4428
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[72]	valid_0's mape: 0.431998
 - fold3 - 0.4320
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[60]	valid_0's mape: 0.444427
 - fold4 - 0.4444
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[118]	valid_0's mape: 0.42907
 - fold5 - 0.4291
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[92]	valid_0's mape: 0.427187
 - fold6 - 0.4272
Training until va

[I 2023-08-24 04:43:14,957] Trial 98 finished with value: 0.4355677616830221 and parameters: {'max_depth': 3, 'n_estimators': 33186, 'learning_rate': 0.08252035658525605, 'num_leaves': 6, 'colsample_bytree': 0.2137190387104861, 'subsample': 0.6467907887327868, 'min_data_in_leaf': 110, 'max_bin': 148, 'reg_lambda': 14.260876811747934}. Best is trial 33 with value: 0.4354671137960793.


Early stopping, best iteration is:
[142]	valid_0's mape: 0.434911
 - fold8 - 0.4349
FINISHI: Whole Score: 0.4356
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[459]	valid_0's mape: 0.429807
 - fold1 - 0.4298
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[88]	valid_0's mape: 0.442716
 - fold2 - 0.4427
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[305]	valid_0's mape: 0.432126
 - fold3 - 0.4321
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[89]	valid_0's mape: 0.443981
 - fold4 - 0.4440
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[318]	valid_0's mape: 0.428747
 - fold5 - 0.4287
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[89]	valid_0's mape: 0.427338
 - fold6 - 0.4273
Training until

[I 2023-08-24 04:43:32,329] Trial 99 finished with value: 0.43551451988838574 and parameters: {'max_depth': 3, 'n_estimators': 40456, 'learning_rate': 0.0740370086179596, 'num_leaves': 6, 'colsample_bytree': 0.2099617716984567, 'subsample': 0.6810317440377563, 'min_data_in_leaf': 104, 'max_bin': 167, 'reg_lambda': 14.02178172367549}. Best is trial 33 with value: 0.4354671137960793.


Early stopping, best iteration is:
[111]	valid_0's mape: 0.435069
 - fold8 - 0.4351
FINISHI: Whole Score: 0.4355
Number of finished trials: 100
Best trial: {'max_depth': 3, 'n_estimators': 3592, 'learning_rate': 0.0525394953750778, 'num_leaves': 5, 'colsample_bytree': 0.17392127287986914, 'subsample': 0.5448823424698281, 'min_data_in_leaf': 263, 'max_bin': 311, 'reg_lambda': 52.94591316469016}
 - fold1 - 0.4300
 - fold2 - 0.4431
 - fold3 - 0.4324
 - fold4 - 0.4441
 - fold5 - 0.4285
 - fold6 - 0.4269
 - fold7 - 0.4438
 - fold8 - 0.4351
FINISHI: Whole Score: 0.4355


In [25]:
lgbm_params = {
    "task":"train",
    "objective": "mape",
    "boosting":"gbdt",
    "metric": 'mape',
    "seed" : 88,
    "importance_type": "gain",
    "random_state": 88,
    "verbose": -1,
    'max_depth': 3,
    'n_estimators': 3592,
    'learning_rate': 0.0525394953750778,
    'num_leaves': 5,
    'colsample_bytree': 0.17392127287986914,
    'subsample': 0.5448823424698281,
    'min_data_in_leaf': 263,
    'max_bin': 311,
    'reg_lambda': 52.94591316469016
}


# training
score, oof_lgb, models_lgb, transformed_test_data_list = fit_lgbm(
    X=df,
    y=df["price"],
    test_data=test,
    categorical_cols=[],
    params=lgbm_params,
    cv=cv,
    verbose=False,
)

 - fold1 - 0.4300
 - fold2 - 0.4431
 - fold3 - 0.4324
 - fold4 - 0.4441
 - fold5 - 0.4285
 - fold6 - 0.4269
 - fold7 - 0.4438
 - fold8 - 0.4351
FINISHI: Whole Score: 0.4355


In [ ]:
Best trial: {'max_depth': 3, 'n_estimators': 3592, 'learning_rate': 0.0525394953750778, 'num_leaves': 5, 'colsample_bytree': 0.17392127287986914, 'subsample': 0.5448823424698281, 'min_data_in_leaf': 263, 'max_bin': 311, 'reg_lambda': 52.94591316469016}
 - fold1 - 0.4300
 - fold2 - 0.4431
 - fold3 - 0.4324
 - fold4 - 0.4441
 - fold5 - 0.4285
 - fold6 - 0.4269
 - fold7 - 0.4438
 - fold8 - 0.4351
==================================================
FINISHI: Whole Score: 0.4355

In [26]:
sub_df = pd.read_csv(CFG.data_dir + "submit_sample.csv", names=["id", "price"])

In [27]:
preds = []
for transformed_test_data, model in zip(transformed_test_data_list, models_lgb):
    pred = model.predict(transformed_test_data)
    preds.append(pred)
pred = np.mean(preds, axis=0)

# submission
sub_df["price"] = pred
sub_df.to_csv(f'/content/drive/MyDrive/Colab Notebooks/signate2023/exp/kun_{CFG.filename}.csv', index=False, header=False)

In [28]:
pd.concat([df['id'], pd.DataFrame(oof_lgb, columns=['oof_pred'])], axis=1).to_csv(f'/content/drive/MyDrive/Colab Notebooks/signate2023/exp/kun_{CFG.filename}_oof_pred.csv', index=False)